# Git Status com API Fabric - Power BI

#### Notebook usando a REST API do Power BI Fabric com a finalidade de listar os workspaces e efetuar um git status com todas as alterações que ainda não tiveram commit git integration. Pandas usado para normalização da saída e gravação em csv.

##### 1. Inicie o ambiente virtual venv - python -m venv .venv
##### 2. Instale os pacotes necessários - pip install -r requirements.txt

In [1]:
# Importar bibliotecas necessárias
import requests
import pandas as pd
import decouple
import msal
import time
import json
from datetime import datetime

In [8]:
# response = requests.get("https://login.microsoftonline.com")
# print(response.status_code)

In [ ]:
# Variáveis de ambiente para Token
config = decouple.AutoConfig('.') # Solução encontrada em https://github.com/HBNetwork/python-decouple/issues/116 para resolver o problema de encontrar o .env

# access_token1 = config('TOKEN1') # Token 1 para listagem dos workspaces
# access_token2 = config('TOKEN2') # Token 2 para o git status

client_id = config('CLIENT_ID')
secret_id = config('CLIENT_SECRET')
tenant_id = config('TENANT_ID')
username = config('USERNAME')
password = config('PASSWORD')

# obter Token por método
def obter_token(username, password, tenant_id, client_id, scopes):
    token_response = None
    app = msal.PublicClientApplication(client_id, authority=tenant_id)
    token_response = app.acquire_token_by_username_password(username, password, scopes)

    if 'access_token' in token_response:
        print('Token obtido com sucesso!')
        return token_response['access_token']
    else:
        print('Erro ao retornar Token:')
        for key, value in token_response.items():
            print(f'{key}: {value}')
        return None

# # Chama a função que gera o token para a variável access_token
access_token = obter_token(
    username= username,
    password= password,
    tenant_id=f"https://login.microsoftonline.com/{tenant_id}",
    client_id= client_id,
    scopes=["https://analysis.windows.net/powerbi/api/.default"],
)

# ConfidentialClientApplication
# msal_app = msal.ConfidentialClientApplication(
#     client_id=config('CLIENT_ID'),
#     client_credential=config('CLIENT_SECRET'),
#     authority=f"https://login.microsoftonline.com/{tenant_id}"
# )
# token_response = msal_app.acquire_token_for_client(scopes=scope)

# if "access_token" in token_response:
#     access_token = token_response["access_token"]
# else:
#     token_response.get("error_description", "Erro desconhecido")


# headers = {
#     "Authorization": f"Bearer {access_token}"
# }
# url = "https://api.powerbi.com/v1.0/myorg/groups"

# response = requests.get(url, headers=headers)
# print(response.status_code)
# print(response.text)

In [3]:
# Método para listar todos os workspaces
def lista_workspaces():
    url = 'https://api.powerbi.com/v1.0/myorg/groups'
    headers = {'Authorization': f'Bearer {access_token}'}
    resposta = requests.get(url, headers=headers)
    resposta.raise_for_status()

    workspaces = resposta.json().get('value', [])
    workspaces_filter = [item for item in workspaces if item.get('capacityId') == 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxx']

    return workspaces_filter

In [ ]:
lista_workspaces()

In [4]:
# Lista usuários por grupo
# GET https://api.powerbi.com/v1.0/myorg/groups/{groupId}/users
def lista_user_workspaces():
    groupId = lista_workspaces()
    itens = []

    for id_ws in groupId:
        url = f'https://api.powerbi.com/v1.0/myorg/groups/{id_ws['id']}/users'
        headers = {'Authorization': f'Bearer {access_token}'}
        resposta = requests.get(url, headers=headers)
        resposta.raise_for_status()
    
        workspaces = resposta.json().get('value', [])

        for rel in workspaces:
                rel['workspace_id'] = id_ws['id']
                rel['workspace_name'] = id_ws.get('name', 'Não encontrado')
                itens.append(rel)
    
    if itens:
        df = pd.json_normalize(itens)
    df.to_csv('lista_user_group.csv', index=False, encoding='iso8859-9')
        print(f'Arquivo csv git_status.csv gerado com sucesso, registros: {len(itens)}')

    return df

In [ ]:
lista_user_workspaces()

In [78]:
# Método para listar atividades dos relatórios
def lista_report30dias():
    groupId = lista_workspaces()
    itens = []
    
    for id_ws in groupId:
        try:
            url = f'https://api.powerbi.com/v1.0/myorg/admin/groups/{id_ws['id']}/unused'
            headers = {'Authorization': f'Bearer {access_token}'}
            resposta = requests.get(url, headers=headers)

            # Tempo limite de requisições da API 30 segundos
            if resposta.status_code == 429:
                repetir = int(resposta.headers.get('Retry-After', 30))
                print(f'Aguardando limite da API {repetir} segundos...')
                time.sleep(repetir)
                continue
            
            resposta.raise_for_status()    
            relatorios = resposta.json().get('unusedArtifactEntities', [])
            
            for rel in relatorios:
                rel['workspace_id'] = id_ws['id']
                rel['workspace_name'] = id_ws.get('name', 'Não encontrado')
                itens.append(rel)

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 400:
                print(f'Erro 400 workspace {id_ws.get('name', id_ws['id'])}')
                continue
            else:
                print(f'Erro de requisição {e.response.status_code} workspace {id_ws.get('name', id_ws['id'])}: {e}')
                continue
        except Exception as e:
            print(f'Erro no workspace {id_ws.get('name', id_ws['id'])}: {e}')
            continue

    if itens:
        df = pd.json_normalize(itens)
        df['data_Accessed'] = pd.to_datetime(df['lastAccessedDateTime'], format='mixed', utc=True)
        df['diasSemUtilizacao'] = datetime.now() - df['data_Accessed']
        df1 = df[['artifactId','displayName','artifactType','artifactSizeInMB',	'createdDateTime','data_Accessed','diasSemUtilizacao','workspace_id','workspace_name']]

        return df1

In [ ]:
lista_report30dias()

In [ ]:
# Retorna o git status da lista de workspaces que tem o git configurado
def git_status():
    # Lista os workspaces e armazena na variável workspaceId
    workspaceId = lista_workspaces()

    status = [] # Lista vazia para armazenar todas as alterações encontradas
    for id_ws in workspaceId:
        try:
            url = f'https://api.fabric.microsoft.com/v1/workspaces/{id_ws['id']}/git/status'
            headers = {'Authorization': f'Bearer {access_token2}'}
            resposta = requests.get(url, headers=headers)

            # Caso o workspace não tenha git o retorno é 400
            if resposta.status_code == 400:
                print(f'Erro 400 workspace {id_ws.get('name', id_ws['id'])}')
                continue
            
            # Tempo limite de requisições da API 60 segundos
            if resposta.status_code == 429:
                repetir = int(resposta.headers.get('Retry-After', 60))
                print(f'Aguardando limite da API {repetir} segundos...')
                time.sleep(repetir)
                continue
            
            resposta.raise_for_status()
            changes = resposta.json().get('changes', [])

            # Imprimindo id e nome de cada workspace da resposta
            for change in changes:
                change['workspace_id'] = id_ws['id']
                change['workspace_name'] = id_ws.get('name', 'Não encontrado')
                status.append(change)

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 400:
                print(f'Erro 400 workspace {id_ws.get('name', id_ws['id'])}')
                continue
            else:
                print(f'Erro de requisição {e.response.status_code} workspace {id_ws.get('name', id_ws['id'])}: {e}')
                continue
        except Exception as e:
            print(f'Erro no workspace {id_ws.get('name', id_ws['id'])}: {e}')
            continue

    # Usando pandas para normalizar nossa saída e gravar csv
    if status:
        df = pd.json_normalize(status)
        df.to_csv('git_status.csv', index=False, encoding='iso8859-9')
        print(f'Arquivo csv git_status.csv gerado com sucesso, registros: {len(status)}')
        return df
    else:
        print('Sem alterações encontradas')
        return pd.DataFrame()

In [ ]:
git_status()

In [ ]:
# Método para listar todos os workspaces e normalizar com Pandas
def lista_workspaces_csv():
    url = 'https://api.powerbi.com/v1.0/myorg/groups'
    headers = {'Authorization': f'Bearer {access_token}'}
    resposta = requests.get(url, headers=headers)
    resposta.raise_for_status()

    workspaces = resposta.json().get('value', [])

    if workspaces:
        df = pd.json_normalize(workspaces)
        df.to_csv('lista_workspaces.csv', index=False)
        print(f'Arquivo lista_workspaces.csv gerado com sucesso, registros: {len(workspaces)}')
        return df
    # return workspaces

In [ ]:
lista_workspaces_csv()

In [ ]:
# Método para retornar todos os relatórios de uma lista de workspaces
def lista_relatorios_ws():
    # Lista os workspaces e armazena na variável workspaceId
    workspaceId = lista_workspaces()

    lista_rel = [] # Lista vazia para armazenar todos os relatórios do workspace
    for id_ws in workspaceId:
        try:
            url = f'https://api.powerbi.com/v1.0/myorg/groups/{id_ws['id']}/reports'
            headers = {'Authorization': f'Bearer {access_token}'}
            resposta = requests.get(url, headers=headers)

            # Caso ocorrá erro no workspace
            if resposta.status_code == 400:
                print(f'Erro 400 workspace {id_ws.get('name', id_ws['id'])}')
                continue

            resposta.raise_for_status()
            values = resposta.json().get('value', [])

            # Imprimindo id e nome de cada workspace da resposta
            for value in values:
                value['workspace_id'] = id_ws['id']
                value['workspace_name'] = id_ws.get('name', 'Não encontrado')
                lista_rel.append(value)

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 400:
                print(f'Erro 400 workspace {id_ws.get('name', id_ws['id'])}')
                continue
            else:
                print(f'Erro de requisição {e.response.status_code} workspace {id_ws.get('name', id_ws['id'])}: {e}')
                continue
        except Exception as e:
            print(f'Erro no workspace {id_ws.get('name', id_ws['id'])}: {e}')
            continue

    # Usando pandas para normalizar nossa saída e gravar csv
    if lista_rel:
        df = pd.json_normalize(lista_rel)
        df.to_csv('lista_relatorios_ws.csv', index=False, encoding='iso8859-9')
        print(f'Arquivo csv lista_relatorios_ws.csv gerado com sucesso, registros: {len(lista_rel)}')
        return df
    else:
        print('Não foram encontrados relatórios')
        return pd.DataFrame()

In [ ]:
lista_relatorios_ws()

#### Normalizar Json do Git log

In [ ]:
# !pip install openpyxl
import codecs
import json
import re
from datetime import datetime
from dateutil import parser

with open('git_log.json', encoding='iso8859-2') as f:
    data = json.load(f)

df = pd.json_normalize(data)

# Método para extrair uuid da coluna message
def extrair_uuid(message):
    message_2 = re.search(r'workspace\s+([a-f0-9\-]{36})', message)
    if message_2:
        return message_2.group(1)
    else:
        return message
        
df['message_1'] = df['message'].apply(extrair_uuid)

# df['data_1'] = df['date'].apply(lambda x: pd.to_datetime(parser.parse(x)))
df['data_1'] = pd.to_datetime(df['date'], format='mixed', utc=True)
df['commit_by_date_1'] = pd.to_datetime(df['commit_by_date'], format='mixed', utc=True)
df['data'] = df['data_1'].dt.strftime('%d/%m/%Y')
df['commit_by_data'] = df['commit_by_date_1'].dt.strftime('%d/%m/%Y')

# Colunas necessárias para o dataframe
df1 = df[['commit','author', 'author_email', 'data', 'commit_by', 'commit_by_email', 'commit_by_data', 'stats.files_changed', 'stats.insertions', 'stats.deletions', 'message_1']]

df1.to_excel('git_log.xlsx', index=False)